In [3]:
import glob
import re
import pathlib
import os
import numpy as np
import pandas as pd

In [ ]:
def uttspkdict(FOLDER_PATH):
    spkrlist = os.listdir(FOLDER_PATH)
    spkrlist.sort()
    uttspk = {}
    for a in range(len(spkrlist)):
        uttspk[spkrlist[a]] = [x for x in os.listdir(FOLDER_PATH+'/'+str(spkrlist[a])) if x.endswith("normal.wav")]
    return uttspk

def uttspkdict2(FOLDER_PATH):
    metadata = pd.read_csv('/home/oem/Desktop/DiCOVA/DiCOVA_Track_2_Release/metadata.csv')
    spkrlist = list(metadata['ID'])
    spkrlist.sort()
    uttspk = {}
    for a in range(len(spkrlist)):
        uttspk[spkrlist[a]] = str(spkrlist[a])+"-counting-normal.wav"
    return uttspk

def writeuttspk(TEXT_PATH,dictionary,mode):
    d_keys = dictionary.keys()
    newlines = []
    for spkr in d_keys:
        for utt in dictionary[spkr]:
            newline = spkr+'-'+utt + ' ' + spkr        
            newlines.append(newline)
    if (mode == 'new'):
        with open(TEXT_PATH,'w') as f:
            f.write("\n".join(newlines))
    elif (mode == 'append'):
        with open(TEXT_PATH,'a') as f:
            f.write("\n")
            f.write("\n".join(newlines))
    else:
        raise ValueError('Mode Not supported')
    return 

def writeuttspk2(TEXT_PATH,dictionary,mode):
    d_keys = dictionary.keys()
    newlines = []
    for spkr in d_keys:
        utt = dictionary[spkr]
        newline = utt + ' ' + spkr        
        newlines.append(newline)
    if (mode == 'new'):
        with open(TEXT_PATH,'w') as f:
            f.write("\n".join(newlines))
    elif (mode == 'append'):
        with open(TEXT_PATH,'a') as f:
            f.write("\n")
            f.write("\n".join(newlines))
    else:
        raise ValueError('Mode Not supported')
    return 


def getwavpath(FOLDER_PATH):
    my_list = list(pathlib.Path(FOLDER_PATH).glob('*normal.wav'))
    my_list = [x.as_posix() for x in my_list]
    my_list.sort()
    return my_list

def gettext(wavpath,asr,mode):
    if(asr == 'aspire'):
        if (mode == 'convert'):
            text = "/usr/bin/sox -t wav " + wavpath + " -c 1 -b 16 -r 8000 -t wav - |"
        elif(mode == 'direct'):
            text = wavpath
    elif(asr == 'timit'):
        if (mode == 'convert'):
            text = "/home/oem/Desktop/kaldi/egs/timit/s5/../../../tools/sph2pipe_v2.5/sph2pipe -f wav " + wavpath + " |"
        elif(mode == 'direct'):
            text = wavpath
    return text

def writescp(spkr,utt,text,dictionary,TEXT_PATH,mode):
    d_keys = dictionary.keys()
    newlines = []
    newline = spkr+'-'+utt + ' ' + text        
    newlines.append(newline)
    if (mode == 'new'):
        with open(TEXT_PATH,'w') as f:
            f.write("\n".join(newlines))
    elif (mode == 'append'):
        with open(TEXT_PATH,'a') as f:
            f.write("\n")
            f.write("\n".join(newlines))
    else:
        raise ValueError('Mode Not supported')
    return 


# Get utt2sp and wav.scp files for timit

In [ ]:
"""
Run this to get utt2spk file for timit
"""

TEXT_PATH = '/home/oem/Desktop/asrworkdir/utt2spk.txt'
FOLDER_PATH = "/home/oem/Desktop/asrworkdir/data/test/dr"

for i in range(1,8):
    my_dict = uttspkdict(FOLDER_PATH+str(i))
    if i == 1:
        writeuttspk(TEXT_PATH,my_dict,'new')
    else:
        writeuttspk(TEXT_PATH,my_dict,'append')

        
"""
Run this to get the wavscp file for timit
"""

SCP_PATH = '/home/oem/Desktop/asrworkdir/wavscp.txt'
FOLDER_PATH = "/home/oem/Desktop/asrworkdir/data/test/dr"

for i in range(1,8):
    my_dict = uttspkdict(FOLDER_PATH+str(i))
    spkrlist = os.listdir(FOLDER_PATH+str(i))
    spkrlist.sort()
    for k in range(len(spkrlist)):
        spkr = spkrlist[k]
        utt = [x for x in os.listdir(FOLDER_PATH+str(i)+"/"+spkrlist[k]) if x.endswith(".wav")]
        utt.sort()
        wavpathlist = getwavpath(FOLDER_PATH+str(i)+"/"+spkrlist[k])
        for j in range(len(wavpathlist)):
            txt = gettext(wavpathlist[j],'timit','convert')
            if(i==1 and k==0 and j==0):
                writescp(spkr,utt[j],txt,my_dict,SCP_PATH,'new')           
            else:
                writescp(spkr,utt[j],txt,my_dict,SCP_PATH,'append')           

# Create utt2spk and wav.scp files for aspire

In [ ]:
FOLDER_PATH = "/home/oem/Desktop/DiCOVA/Coswara/counting-normal"
sub = [ f.path for f in os.scandir(FOLDER_PATH)]
sub.sort()

In [ ]:
"""
Run this to get utt2spk file for aspire
"""

TEXT_PATH = 'utt2spk'
FOLDER_PATH = "/home/oem/Desktop/DiCOVA/Coswara/counting-normal"
my_dict = uttspkdict2(FOLDER_PATH)
writeuttspk2(TEXT_PATH,my_dict,'append')          

In [ ]:
"""
Run this to get the wavscp file for aspire
"""
SCP_PATH = '/home/oem/Desktop/DiCOVA/Coswara/wav.scp'
wavpathlist = []
my_dict = uttspkdict2(FOLDER_PATH)
metadata = pd.read_csv('/home/oem/Desktop/DiCOVA/DiCOVA_Track_2_Release/metadata.csv')
spkrlist = list(metadata['ID'])
spkrlist.sort()
for k in range(len(spkrlist)):
    spkr = spkrlist[k]
    utt = "counting-normal.wav"
    print(utt)
    print(sub[k])
    wavpathlist.append(sub[k])
    txt = gettext(wavpathlist[k],'aspire','convert')
    if(k==0):
        writescp(spkr,utt,txt,my_dict,SCP_PATH,'new')           
    else:
        writescp(spkr,utt,txt,my_dict,SCP_PATH,'append') 

# Format the posteriors files suitable to the next code

In [ ]:
with open("/home/oem/Desktop/DiCOVA/posteriors/combined/post4.txt") as f:
    lines = [line.rstrip() for line in f]

In [ ]:
import os
Post = "/home/oem/Desktop/DiCOVA/posteriors/"
Post_dir = Post+"counting-normal/"

if os.path.isdir(folder):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(Post_dir)

print(len(lines))
for i in range(len(lines)):
    x = lines[i]
    y = x[:25]+"post"
    with open(Post_dir+y,'a') as f:
        z = x
        a = z.replace(" [",",[")
        f.write(a)

# Get Posterior matrices for each utterance

In [2]:
def findphoneindexprob(filename,phone_index):
    f0 = open(filename,'r')
    array = f0.read().split(',')
    arr_indexprob=[]
    for i in range (1,len(array)):
        s0 = array[i]
        if  ' '+str(phone_index) + ' ' in s0:
            k=0
            start = ' '+str(phone_index)+' '
            s = array[i]
            p = s.find(start)+len(start)
            for space in s[p:]:
                if (space == ' '):
                    break
                k+=1
            arr_indexprob += [float(s[s.find(start)+len(start):s.find(start)+len(start)+k])]

        else:
            arr_indexprob += [0.0]
    return arr_indexprob

def findphoneprob(filename,phone):
    with open("phonemap.txt", 'r') as document:
        phonedict = {}
        for line in document:
            line = line.split()
            phonedict[line[0]] = line[1:]
    arr_phoneprob = np.zeros(len(findphoneindexprob(filename,200))) #200 is an arbitrary non phone_index
    for i in phonedict[phone]:
        arr_phoneprob = np.add(findphoneindexprob(filename,i),arr_phoneprob)
    return arr_phoneprob

def sentenceprob(filename):
    f = open("phonenames.txt",'r')
    phonenamearray = f.read().split(',')
    y = []
    for phone in phonenamearray:
        y += [findphoneprob(filename,phone)]
    y = np.array(y)    
    return y.T

# Get the MFCC files in npy format from the scp file

In [ ]:
import kaldiio
import numpy as np
PARENT_DIR = "/home/oem/Desktop/DiCOVA"
FOLDER_PATH = "/home/oem/Desktop/DiCOVA/Coswara/counting-normal"
with open(FOLDER_PATH+"/feats.scp",'r') as f:
    for line in f:
        (key,rxfile) = line.split(' ')
        mfcc = kaldiio.load_mat(rxfile)
        np.save(PARENT_DIR+"/feats/mfcc/"+key,mfcc)

# Get the First Order statistics

In [1]:
def fstat(spkr,output_loc,mfcc_data,postprob_data,phoneclass,threshold,start,shift,framelength):
    for key in phoneclass:
        while ((start+framelength)<=len(mfcc_data)):
            mfcc = mfcc_data[start:(start+framelength)]
            postprob = postprob_data[start:(start+framelength)]
            postprob1 = postprob[:,phoneclass[key]]
            if(np.sum(np.sum(postprob1,axis=0)) > threshold):
                phonesum = np.sum(postprob1,axis = 0)
                for k in range(0,len(phonesum)):
                    if (phonesum[k] == 0):
                        phonesum[k] = 1e-8
                norm = np.divide(postprob1,phonesum)
                vector1 = (np.dot(norm.T,mfcc))
                vector = vector1.flatten()
                np.save(output_loc+key+'/'+spkr+'_'+str(start)+'.npy',vector)                    

            start+=shift    
    return

In [5]:
phoneclass = {'full':[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38]}
mfcc_dir = '/home/oem/Desktop/DiCOVA/feats/mfccnew/'
mfcc = os.listdir(mfcc_dir)
mfcc.sort()
post_dir = '/home/oem/Desktop/DiCOVA/posteriors/counting-normal/'
post = os.listdir(post_dir)
post.sort()
output_loc = '/home/oem/Desktop/DiCOVA/stats/' 
for i in range(len(mfcc)):
        mfcc_data = np.load(mfcc_dir+mfcc[i])
        postprob_data = sentenceprob(post_dir+post[i])
        threshold = 0
        start = 0
        shift = len(mfcc_data)
        framelength = len(mfcc_data)
        spkr = post[i][:-5]
        fstat(spkr,output_loc,mfcc_data,postprob_data,phoneclass,threshold,start,shift,framelength)

In [ ]:
output_loc = '/home/oem/Desktop/DiCOVA/stats/' 
x = np.load('/home/oem/Desktop/DiCOVA/stats/augmented/AATVTZch-counting-normal_10.npy')
np.shape(x)

# Apply PCA to the Features

In [7]:
metadata = pd.read_csv('/home/oem/Desktop/DiCOVA/DiCOVA_Track_2_Release/metadata.csv')
metadata = metadata.replace('p',1)
md = metadata.replace('n',0)

fstats_folder = '/home/oem/Desktop/DiCOVA/stats/augmented/'
fstats = os.listdir(fstats_folder)
fstats.sort()
x = np.fromfile(fstats_folder+fstats[1])
md['ID'][0],md['Covid_status'][0]

('AANHmWlD', 0)

In [8]:
fstats_folder = '/home/oem/Desktop/DiCOVA/stats/full/'
x_folder = '/home/oem/Desktop/DiCOVA/'
fstats = os.listdir(fstats_folder)
fstats.sort()
for i in range(1,6):
    file = open('/home/oem/Desktop/DiCOVA/DiCOVA_Track_2_Release/LISTS/val_fold_'+str(i)+'.txt', 'r')
    ids = file.readlines()
    ids = [line.strip() for line in ids]
    ids.sort()
    for j in range(len(fstats)):
        fstat = fstats[j]
        ID = fstat[:8]
        for k in range(len(ids)):
            if(ID == ids[k]):
                fn = list(md['ID'])
                ind = fn.index(ID)
                label = md['Covid_status'][ind]
                text = fstats_folder+fstat+" "+str(label)
                with open(x_folder+"val_fold_"+str(i)+".list",'a') as listfile:
                    listfile.write(text+"\n")

In [10]:
import numpy as np
featdim =1560
def load_input(file_path):
    with open(file_path) as f:  
        line = f.readline()
        cnt = 1
        tr = np.empty(shape=[0,featdim])
        covid_trgt = []
        while line:
            [path,rem] = line.split()
            covid = float(rem)
            data_array = np.load(path)
            da = np.reshape(data_array,(1,-1))
            covid_trgt.append(height)
            tr = np.append(tr,da,axis=0)
            line = f.readline()
            cnt+=1
        print("train shape :",np.shape(tr), "targets shape :", np.shape(covid_trgt))
    return [tr,covid_trgt]

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


tr1_X,tr1_Y = load_input("/home/oem/Desktop/DiCOVA/train_fold_1.list")
tr2_X,tr2_Y = load_input("/home/oem/Desktop/DiCOVA/train_fold_2.list") 
tr3_X,tr3_Y = load_input("/home/oem/Desktop/DiCOVA/train_fold_3.list") 
tr4_X,tr4_Y = load_input("/home/oem/Desktop/DiCOVA/train_fold_4.list") 
tr5_X,tr5_Y = load_input("/home/oem/Desktop/DiCOVA/train_fold_5.list") 

val1_X,val1_Y = load_input("/home/oem/Desktop/DiCOVA/val_fold_1.list")
val2_X,val2_Y = load_input("/home/oem/Desktop/DiCOVA/val_fold_2.list")
val3_X,val3_Y = load_input("/home/oem/Desktop/DiCOVA/val_fold_3.list")
val4_X,val4_Y = load_input("/home/oem/Desktop/DiCOVA/val_fold_4.list")
val5_X,val5_Y = load_input("/home/oem/Desktop/DiCOVA/val_fold_5.list")


train shape : (15369, 1560) targets shape : (15369,)
train shape : (15652, 1560) targets shape : (15652,)
train shape : (15734, 1560) targets shape : (15734,)
train shape : (15748, 1560) targets shape : (15748,)
train shape : (15789, 1560) targets shape : (15789,)
train shape : (207, 1560) targets shape : (207,)
train shape : (207, 1560) targets shape : (207,)
train shape : (207, 1560) targets shape : (207,)
train shape : (207, 1560) targets shape : (207,)
train shape : (207, 1560) targets shape : (207,)


In [32]:
scaler = StandardScaler()# Fit on training set only.
pca = PCA(300)

scaler.fit(tr1_X)
tr1_X = scaler.transform(tr1_X)
val1_X = scaler.transform(val1_X)
pca.fit(tr1_X)
tr1_X = pca.transform(tr1_X)
val1_X = pca.transform(val1_X)

scaler.fit(tr2_X)
tr2_X = scaler.transform(tr2_X)
val2_X = scaler.transform(val2_X)
pca.fit(tr2_X)
tr2_X = pca.transform(tr2_X)
val2_X = pca.transform(val2_X)

scaler.fit(tr3_X)
tr3_X = scaler.transform(tr3_X)
val3_X = scaler.transform(val3_X)
pca.fit(tr3_X)
tr3_X = pca.transform(tr3_X)
val3_X = pca.transform(val3_X)

scaler.fit(tr4_X)
tr4_X = scaler.transform(tr4_X)
val4_X = scaler.transform(val4_X)
pca.fit(tr4_X)
tr4_X = pca.transform(tr4_X)
val4_X = pca.transform(val4_X)

scaler.fit(tr5_X)
tr5_X = scaler.transform(tr5_X)
val5_X = scaler.transform(val5_X)
pca.fit(tr5_X)
tr5_X = pca.transform(tr5_X)
val5_X = pca.transform(val5_X)

(207, 300)


In [35]:
from sklearn import svm
from sklearn.metrics import confusion_matrix as conf
# clf = svm.LinearSVC(random_state = 0,tol=1e-5,dual=False,penalty='l2',max_iter = 10000,C = 10,class_weight = 'balanced')
C = [0.001,0.01,0.1,1,10,100]
for i in range(len(C)):
    clf = svm.SVC(C = C[i])
    clf.fit(tr1_X,tr1_Y)
    acc1=clf.score(val1_X,val1_Y)
    y_pred1 = clf.predict(val1_X)

    clf.fit(tr2_X,tr2_Y)
    acc2=clf.score(val2_X,val2_Y)
    y_pred2 = clf.predict(val2_X)

    clf.fit(tr3_X,tr3_Y)
    acc3=clf.score(val3_X,val3_Y)
    y_pred3 = clf.predict(val3_X)

    clf.fit(tr4_X,tr4_Y)
    acc4=clf.score(val4_X,val4_Y)
    y_pred4 = clf.predict(val4_X)

    clf.fit(tr5_X,tr5_Y)
    acc5=clf.score(val5_X,val5_Y)
    y_pred5 = clf.predict(val5_X)
    
    conf1 = conf(val1_Y,y_pred1)
    conf2 = conf(val2_Y,y_pred2)
    conf3 = conf(val3_Y,y_pred3)
    conf4 = conf(val4_Y,y_pred4)
    conf5 = conf(val5_Y,y_pred5)
    
    print(C[i],conf1)
    print(C[i],conf2)
    print(C[i],conf3)
    print(C[i],conf4)
    print(C[i],conf5)

0.001 [[186   0]
 [ 21   0]]
0.001 [[186   0]
 [ 21   0]]
0.001 [[186   0]
 [ 21   0]]
0.001 [[186   0]
 [ 21   0]]
0.001 [[186   0]
 [ 21   0]]
0.01 [[186   0]
 [ 21   0]]
0.01 [[186   0]
 [ 21   0]]
0.01 [[186   0]
 [ 21   0]]
0.01 [[186   0]
 [ 21   0]]
0.01 [[186   0]
 [ 21   0]]
0.1 [[186   0]
 [ 21   0]]
0.1 [[186   0]
 [ 21   0]]
0.1 [[186   0]
 [ 21   0]]
0.1 [[186   0]
 [ 21   0]]
0.1 [[186   0]
 [ 21   0]]
1 [[186   0]
 [ 21   0]]
1 [[186   0]
 [ 21   0]]
1 [[186   0]
 [ 21   0]]
1 [[186   0]
 [ 21   0]]
1 [[186   0]
 [ 21   0]]
10 [[186   0]
 [ 21   0]]
10 [[186   0]
 [ 21   0]]
10 [[186   0]
 [ 21   0]]
10 [[186   0]
 [ 21   0]]
10 [[186   0]
 [ 21   0]]
100 [[186   0]
 [ 21   0]]
100 [[186   0]
 [ 21   0]]
100 [[186   0]
 [ 21   0]]
100 [[186   0]
 [ 21   0]]
100 [[186   0]
 [ 21   0]]
